In [ ]:
%%writefile requirements.txt
cvxpy>=1.2.2
matplotlib>=3.2.0
numpy>=1.12.0
pandas>=0.19
scikit-learn>=0.24.1
scipy>=1.3.0
yfinance>=0.1.55

Writing requirements.txt


In [ ]:
# !pip install -r requirements.txt
!pip install yfinance
!pip install pendulum
!pip install matplotlib
!pip install pandas-datareader
!pip install pyfolio
!pip install cvxpy
!pip install -r requirements.txt
!pip install PyPortfolioOpt
!pip install openai
!pip install zipline

  Using cached pendulum-2.1.2-cp310-cp310-manylinux_2_35_x86_64.whl
  Using cached pytzdata-2020.1-py2.py3-none-any.whl (489 kB)
  Using cached pyfolio-0.9.2-py3-none-any.whl
  Using cached empyrical-0.5.5-py3-none-any.whl
  Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)
  Using cached pyportfolioopt-1.5.5-py3-none-any.whl (61 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.8/220.8 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
  Using cached zipline-1.4.1.tar.gz (5.4 MB)
  Pre

In [ ]:
import copy
import cvxpy
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib as mpl
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib import mlab
%matplotlib inline
import numpy as np
import pandas as pd
import pyfolio as pf
from pypfopt import CLA
from pypfopt import objective_functions
from pypfopt import plotting
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt.efficient_frontier import EfficientCVaR
from pypfopt.efficient_frontier import EfficientFrontier
import seaborn as sns
from scipy import stats
from scipy.optimize import Bounds
from scipy.optimize import LinearConstraint
from scipy.optimize import minimize
import statistics
import time
import yfinance as yf
import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.10/dist-packages/pyfolio/pos.py:26: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  warnings.warn(


In [ ]:
cvxpy.installed_solvers()

['CVXOPT', 'ECOS', 'ECOS_BB', 'GLPK', 'GLPK_MI', 'OSQP', 'SCIPY', 'SCS']

In [ ]:
import yfinance as yf
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import statistics
from matplotlib import mlab

In [ ]:
ticker_list = ['HBLPOWER.NS', 'SCI.NS', 'POLYCAB.NS']

In [ ]:
monthly_data = yf.download(tickers= ticker_list, start = '2006-06-01', end = '2023-11-26', interval = '1mo')['Adj Close']

[*********************100%%**********************]  3 of 3 completed


In [ ]:
monthly_data

,HBLPOWER.NS,POLYCAB.NS,SCI.NS
Date,,,
2006-06-01,NaN,NaN,71.249290
2006-07-01,NaN,NaN,66.072723
2006-08-01,NaN,NaN,76.009644
2006-09-01,NaN,NaN,83.241226
2006-10-01,NaN,NaN,86.076637
...,...,...,...
2023-07-01,191.419525,4816.799805,103.449486
2023-08-01,267.687836,5123.049805,129.860001
2023-09-01,266.440002,5343.049805,145.506989


In [ ]:
monthly_data.dropna(how = 'any', inplace = True)
ticker_mapping = {ticker: ticker.split('.')[0] for ticker in ticker_list}
monthly_data.rename(columns = ticker_mapping, inplace = True)
monthly_data

,HBLPOWER,POLYCAB,SCI
Date,,,
2019-05-01,24.095266,592.995850,31.992229
2019-06-01,21.523834,603.868408,30.622536
2019-07-01,18.619072,546.806763,28.078812
2019-08-01,14.333351,599.838379,27.393963
2019-09-01,17.666687,668.823425,42.313889
2019-10-01,16.471256,834.748657,56.500046
2019-11-01,14.969464,901.831055,59.190525
2019-12-01,15.357024,968.181580,60.658058
2020-01-01,18.409050,960.668335,58.652431


In [ ]:
risk_free_rate = 0.065

In [ ]:
change = expected_returns.mean_historical_return(monthly_data)/100
change

HBLPOWER    2428.333096
POLYCAB      266.441417
SCI            8.057253
dtype: float64

In [ ]:
std_dev = monthly_data.std()/100
std_dev # devide by 100 to get the % changes above

HBLPOWER     0.745233
POLYCAB     12.975440
SCI          0.362630
dtype: float64

In [ ]:
def calculate_sharpe_ratios(expected_returns, risk_free_rates, standard_deviations):
  sharpe_ratios = (expected_returns - risk_free_rates)/standard_deviations
  return sharpe_ratios

sharpe_ratios = calculate_sharpe_ratios(change, risk_free_rate, std_dev)
sharpe_ratios

HBLPOWER    3258.400145
POLYCAB       20.529278
SCI           22.039686
dtype: float64

In [ ]:
equal_weight_stocks = sharpe_ratios.nlargest(3).index
equal_weight_stocks

Index(['HBLPOWER', 'SCI', 'POLYCAB'], dtype='object')

In [ ]:
# 1 Equal weightage portfolio
# weights calculation below
numstocks = len(equal_weight_stocks)
equal_weightage=np.repeat(1/numstocks, numstocks)
equal_weightage

array([0.33333333, 0.33333333, 0.33333333])

In [ ]:
eq_expected_returns = change[equal_weight_stocks]
eq_expected_returns

HBLPOWER    2428.333096
SCI            8.057253
POLYCAB      266.441417
dtype: float64

In [ ]:
eq_expected_returns = np.dot(equal_weightage, eq_expected_returns)
eq_expected_returns

900.9439222744023

In [ ]:
risk= np.std(eq_expected_returns)
risk

0.0